In [1]:
import numpy as np
import pandas as pd
# pd.options.mode.chained_assignment = None  # default='warn'
import re

from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from urllib import request,response

# ***********************************PLEASE READ***********************************
#### This will open an extra chrome tab just used to parse some of the data that I couldnt get normally from the website directly because it parsed back "None" Values for some elements even though it existed, therefor I couldn't get all the data I wanted to collect for this project. the chrome tab allows me read those without restrictions and it solves the "None" Value problem

## IMPORTANT: 
### 1) Please dont close the tab or the project will break, you can close the tab when you stop running the code.
### 2) If you wish, you can rerun the code and it will go scrap the website again and update the dataframe incase new games have been added since the creation of this project.

--------------------------------------------------------------------------
<h3> 1. Web scrapping to collect the Data

In [51]:
"""
*****************************************************INSTRUCTIONS*****************************************************
This will open an extra chrome tab just used to parse some of the data that I couldnt get normally from the website 
directly because it parsed back "None" Values for some elements even though it existed, therefor I couldn't get all the data 
I wanted to collect for this project. the chrome tab allows me read those without restrictions and it solves 
the "None" Value problem

IMPORTANT: Please dont close the tab or the project will break, you can close the tab when the console displays 
"Scrapping Complete.".
"""
driver = webdriver.Chrome()
url = 'https://www.imdb.com/search/title/?title_type=video_game&sort=user_rating,desc'
# driver.maximize_window()
driver.get(url)
content = driver.page_source.encode('utf-8').strip()
""""""
soup = BeautifulSoup(content,"html.parser")
#creates pandas dataframe 
def data_create():
    data = {
        'title': [],
        'release_date': [],
        'maturity_rating': [],
        'genre': [],
        'rating': [],
        'directors': [],
        'stars': [],
        'votes': []
    }
    global df
    df = pd.DataFrame(data= data)
data_create()


#func used to extract data by inputting the soup, the thing you want to find => "selector", and class => "class_selector"
def extract_data(soup, selector, class_selector):
    try:
        return soup.find(selector, class_= class_selector).text.strip()
    except AttributeError:
        return None

#continues looping till its done with finding a "next" link
try: 
    while True:
        #a big section that contains all the data of the videogame title
        game_info = soup.find_all("div", class_="lister-item-content")

        for i in game_info:
            #gets game name
            try:
                game_name = (i.find("h3", class_="lister-item-header")).a.text.strip()
            except AttributeError:
                game_name = None

            #gets release date
            try:
                game_release_date = extract_data(i, "span", "lister-item-year text-muted unbold")
                game_release_date = game_release_date.replace("(", "")
                game_release_date = game_release_date.replace(")", "")
                game_release_date = game_release_date.replace("I", "")
                game_release_date = int(game_release_date.replace("Video Game", "").strip())
            except:
                game_release_date = 0
            
            #gets maturity rating
            game_maturity = extract_data(i, "span", "certificate")

            #gets genre and split it into an array
            try:
                game_genre = extract_data(i, "span", "genre")
                if game_genre.find(",")!=-1:
                    game_genre = np.array(list(game_genre.split(",")))
                else:
                    game_genre = np.array(game_genre)
            except AttributeError:
                game_genre = None

            #get rating
            try:
                game_rating = float(i.find("div", class_="inline-block ratings-imdb-rating").strong.text.strip())
            except AttributeError:
                game_rating = 0

            #get Director & Stars and will split them, and will put "None" in values that dont exist
            #get both
            game_director_and_stars = extract_data(i, "p", "")
            game_director = None
            game_stars = None
            #if both director and star data exist for the title
            if game_director_and_stars.find("|")!=-1:
                game_director_full_text, separator, game_stars_full_text = game_director_and_stars.partition("|")

                game_director_full_text = game_director_full_text.strip()
                game_director_full_text = ''.join(game_director_full_text.splitlines())
                game_stars_full_text = game_stars_full_text.strip()
                game_stars_full_text = ''.join(game_stars_full_text.splitlines())

                director_text, separator, game_director = game_director_full_text.partition(":")
                star_text, separator, game_stars = game_stars_full_text.partition(":")

                #finds if there are many Directors/Stars or just one
                #there are multiple Directors
                if game_director.find(",")!=1:
                    game_director = np.array(list(game_director.split(",")))
                #there is only 1 Director
                else:
                    game_director = np.array(list(game_director))
                #there are multiple Stars
                if game_stars.find(",")!=1:
                    game_stars = np.array(list(game_stars.split(",")))
                #there is only 1 Star
                else:
                    game_stars = np.array(list(game_stars))

            #didnt both Director and Star together, will try to find if only one of them exists        
            #if found Director
            elif game_director_and_stars.find("Director")!=-1:
                game_director_and_stars = ''.join(game_director_and_stars.splitlines())
                director_text, separator, game_director = game_director_and_stars.partition(":")
                game_director = game_director.replace("\n", "")
                #finds if there are many Directors or just one
                #there are multiple Directors
                if game_director.find(",")!=1:
                    game_director = np.array(list(game_director.split(",")))
                #there is only 1 Director
                else:
                    game_director = np.array(list(game_director))
                game_stars = None

            #if found Star
            elif game_director_and_stars.find("Star")!=-1:
                star_text, separator, game_stars = game_director_and_stars.partition(":")
                game_stars = game_stars.replace("\n", "")
                #finds if there are many Stars or just one
                #there are multiple Stars
                if game_stars.find(",")!=1:
                    game_stars = np.array(list(game_stars.split(",")))
                #there is only 1 Star
                else:
                    game_stars = np.array(list(game_stars))
                game_director = None
            else:
                game_director = None
                game_stars = None


            #get votes
            try: 
                game_votes = extract_data(i, "p", "sort-num_votes-visible")
                game_votes = ''.join(game_votes.splitlines())
                deleted_text, separator, game_votes = game_votes.partition(":")
                game_votes = game_votes.replace(",", "").strip()
                game_votes = int(game_votes)
            except:
                game_votes = 0

            #adds game details to dataframe
            df.loc[len(df.index)] = [game_name, game_release_date, game_maturity, game_genre, game_rating, game_director, game_stars, game_votes]
    #         display(df)


        #find button to click on next page 
        uncompleted_next_page_url = soup.find("a", class_="lister-page-next next-page").get("href")
        completed_next_page_url = "https://www.imdb.com/" + uncompleted_next_page_url

        print(completed_next_page_url)

        #replaces old url to completed_next_page_url
        url = completed_next_page_url
        driver.get(url)
        content = driver.page_source.encode('utf-8').strip()

        soup = BeautifulSoup(content, "html.parser")

    driver.quit()
except Exception as e:
    print(e)
    print("Scrapping Complete.")

D:\Anaconda\lib\site-packages\pandas\core\dtypes\cast.py:948: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=51&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=101&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=151&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=201&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=251&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=301&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=351&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=401&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=451&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating

KeyboardInterrupt: 

In [98]:
df

,title,release_date,maturity_rating,genre,rating,directors,stars,votes
0,The Last of Us: Part I,2022,M,['Action' ' Adventure' ' Drama'],9.8,['Matthew Gallant' ' Bruce Straley'],['Ashley Johnson' ' Troy Baker' ' Hana Hayes' ...,3525
1,The Witcher 3: Wild Hunt - Blood and Wine,2016,M,['Action' ' Adventure' ' Drama'],9.8,['Konrad Tomaszkiewicz'],['Doug Cockle' ' MyAnna Buring' ' Antonia Bern...,8795
2,The Last of Us,2013,M,['Action' ' Adventure' ' Drama'],9.7,['Neil Druckmann' ' Bruce Straley'],['Troy Baker' ' Ashley Johnson' ' Hana Hayes' ...,69017
3,God of War: Ragnarök,2022,M,['Action' ' Adventure' ' Drama'],9.7,['Eric Williams'],['Christopher Judge' ' Sunny Suljic' ' Daniell...,11569
4,Red Dead Redemption II,2018,M,['Action' ' Adventure' ' Crime'],9.7,NaN,['Benjamin Byron Davis' ' Roger Clark' ' Rob W...,42853
...,...,...,...,...,...,...,...,...
33519,Through the Winter,2023,NaN,NaN,0.0,['Michael Smith'],['Rita Amparita' ' Hannah Beard' ' Xander M Gr...,0
33520,Atari 50: The Anniversary Celebration,2022,NaN,['Documentary' ' Action' ' Adventure'],0.0,NaN,['Al Alcorn' ' Nolan Bushnell' ' David Crane'],0
33521,Yars Revenge Enhanced,2022,NaN,['Action' ' Fantasy' ' Sci-Fi'],0.0,NaN,NaN,0
33522,Our Summers,2022,NaN,Romance,0.0,['Wavi'],['Reece Bridger'],0


In [2]:
#Save CSV (still has errors)
df.to_csv('CSVs/1) beforeCleaningCSV.csv', index=False)

NameError: name 'df' is not defined

<b>---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------<b>

<h3> 2. Analysis and Data Cleaning

In [88]:
#Import the CSV with errors still
df1 = pd.read_csv("CSVs/1) beforeCleaningCSV.csv")
df1

,title,release_date,maturity_rating,genre,rating,directors,stars,votes
0,The Witcher 3: Wild Hunt - Blood and Wine,2016,M,['Action' ' Adventure' ' Drama'],9.8,['Konrad Tomaszkiewicz'],['Doug Cockle' ' MyAnna Buring' ' Antonia Bern...,8795
1,The Last of Us,2013,M,['Action' ' Adventure' ' Drama'],9.7,['Neil Druckmann' ' Bruce Straley'],['Troy Baker' ' Ashley Johnson' ' Hana Hayes' ...,69017
2,God of War: Ragnarök,2022,M,['Action' ' Adventure' ' Drama'],9.7,['Eric Williams'],['Christopher Judge' ' Sunny Suljic' ' Daniell...,11569
3,Red Dead Redemption II,2018,M,['Action' ' Adventure' ' Crime'],9.7,NaN,['Benjamin Byron Davis' ' Roger Clark' ' Rob W...,42853
4,The Witcher 3: Wild Hunt,2015,M,['Action' ' Adventure' ' Drama'],9.7,['Konrad Tomaszkiewicz'],['Doug Cockle' ' Denise Gough' ' Jo Wyatt' ' J...,29120
...,...,...,...,...,...,...,...,...
33518,Through the Winter,2023,NaN,NaN,0.0,['Michael Smith'],['Rita Amparita' ' Hannah Beard' ' Xander M Gr...,0
33519,Atari 50: The Anniversary Celebration,2022,NaN,['Documentary' ' Action' ' Adventure'],0.0,NaN,['Al Alcorn' ' Nolan Bushnell' ' David Crane'],0
33520,Yars Revenge Enhanced,2022,NaN,['Action' ' Fantasy' ' Sci-Fi'],0.0,NaN,NaN,0
33521,Our Summers,2022,NaN,Romance,0.0,['Wavi'],['Reece Bridger'],0


In [82]:
df1.columns

Index(['title', 'release_date', 'maturity_rating', 'genre', 'rating',
       'directors', 'stars', 'votes'],
      dtype='object')

In [83]:
df1.dtypes

title               object
release_date         int64
maturity_rating     object
genre               object
rating             float64
directors           object
stars               object
votes                int64
dtype: object

<b> Looking for Wrong Data<br>
    a: remove untitled upcoming games

In [89]:
for i in df1["title"]:
    get_letter_index = i.find("Untitled")
    if get_letter_index != -1:
        untitled_game_to_be_deleted = i
        
        #gets index of games with "untitled" in them
        get_title_index = df1.index[df1['title']== untitled_game_to_be_deleted].tolist()
        print(get_title_index, i)
        
        #if the game is not called Untitled Goose Game, delete it (we wont delete this one because its the name of an actual game)
        if i != "Untitled Goose Game":
            df1 = df1.drop(get_title_index)
            print("removed")
        else:
            print("The goose lives forever.")
print("Done")

[4639] Untitled Goose Game
The goose lives forever.
[14542] Untitled Iron Man game
removed
[15205] Untitled Indiana Jones Video Game
removed
[15614] Untitled Yakuza: Like A Dragon Sequel
removed
[17646] Untitled Haven PlayStation Game
removed
[19431] Untitled Pokèmon Videogame
removed
[19818] Untitled Tintin game
removed
[20618] Untitled Kaikan Phrase Game App
removed
[20811] Untitled Microsoft IP
removed
[23040] Untitled Mistwalker-Silicion Studio Game
removed
[24352] Untitled Hangman Game
removed
[24646] Untitled the X Collective Project
removed
[25042] Untitled Gaint Camel Game
removed
[28258] Untitled Clam Game
removed
[31458] Untitled Gun Game
removed
Done


In [90]:
#checks that the games are indeed deleted (only Untitled Goose Game will be there)
for i in df1["title"]:
    title_index = i.find("Untitled")
    if title_index != -1:
        untitled_game_to_be_deleted = i
        print(untitled_game_to_be_deleted)
        

Untitled Goose Game


In [95]:
#Save CSV after Title date cleaning 
df1.to_csv('CSVs/2) titleCleaned.csv', index=False)

<b> b: replace release_date that is 0 with the actual date from google

In [96]:
df1["release_date"].unique()

array([2016, 2013, 2022, 2018, 2015, 2021, 1998, 2023, 2010, 2019, 2004,
       2003, 1997, 1995, 2005, 2011, 2009, 2017, 2001, 2008, 2020, 2000,
       1999, 2002, 2014, 1991, 2007, 1990, 2012, 1988, 1994, 1996, 2006,
       1993, 1992, 1985, 1980, 1989, 1987, 1986, 1984, 1969, 1974, 1982,
       1983, 1975, 1981, 1971, 1972,    0, 1978, 1973, 1948, 1968, 1977,
       1979, 1976, 1952, 1958, 1962, 1970, 2024, 2027, 1950, 2030, 1964,
       1967, 2025, 2026], dtype=int64)

In [98]:
#gets index of wrong date data
wrong_date_index = np.array(list(df1.index[df1["release_date"] == 0]))
#gets title of wrong date data
wrong_date_title = np.array(list(df1["title"].loc[df1["release_date"] == 0]))

#puts index besides the name of game
wrong_data_combined = np.stack((wrong_date_index, wrong_date_title), axis=1)

print(f"Length: {len(wrong_data_combined)}")
print()
print(wrong_data_combined)

Length: 612

[['4169' 'Total War: Three Kingdoms']
 ['5151' 'Deliver Us the Moon']
 ['5638' 'Super Street Fighter IV: 3D Edition']
 ...
 ['33419' 'Ion']
 ['33437' 'Never Say Die']
 ['33498' "Devil's Liminal"]]


<b> Cleans the release_date data using Google Search

In [99]:
d = {}

for i in range(len(df1["title"].loc[df1["release_date"] == 0])):
    game_name = wrong_data_combined[i][1]
    searchterm = f"video game {game_name} release date" # edit me

    url = r"https://www.google.com/search?q=" + searchterm #will put what I want to search after q=
    driver = webdriver.Chrome()
    driver.get(url)
    content = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(content,"html.parser")
    
    #will explain its use downwards 
    game_index = wrong_data_combined[i][0]
    
    try:
        # 1) if full date is available as Big Header, need to separate year from the rest
        unfiltered_game_year = soup.find("div", class_="Z0LcW").text.strip()
        unfiltered_years_list = re.findall('\d+', unfiltered_game_year)

        #changes the values inside list into integers
        for i in range(len(unfiltered_years_list)):
            unfiltered_years_list[i] = int(unfiltered_years_list[i])

        #filters numbers between 1930 and 2025
        for i in range(10):
            for j in unfiltered_years_list:
                if j < 1930 or j >= 2025:
                    unfiltered_years_list.remove(j)
        game_year = max(unfiltered_years_list)
        
        #use game index to put in dictionary with the date found from scraping, this is so we can replace the new date in df1
        #for whatever reason it seems that i can only replace the data of the row if i do it manually or if it is in a dictionary, having it in an array doesn't work
        d[game_index] = game_year
        
        #gets last key in dictionary (which is the index of the game in our case)
        game_index = next(reversed(d.keys()))
        game_index = int(game_index)
        
        df1["release_date"][game_index] = game_year
        
        print(url)
        print(game_year)
    
    except:
        # 2) if there is alot of text in Big Header
        try:
            unfiltered_game_year = soup.find("div", class_="wDYxhc").text.strip()
            unfiltered_years_list = re.findall('\d+', unfiltered_game_year)

            #changes the values inside list into integers
            for i in range(len(unfiltered_years_list)):
                unfiltered_years_list[i] = int(unfiltered_years_list[i])

            #filters numbers between 1930 and 2025
            for i in range(10):
                for j in unfiltered_years_list:
                    if j < 1930 or j >= 2025:
                        unfiltered_years_list.remove(j)
            game_year = max(unfiltered_years_list)
            
            #use game index to put in dictionary with the date found from scraping, this is so we can replace the new date in df
            #for whatever reason it seems that i can only replace the data of the row if i do it manually or if it is in a dictionary, having it in an array doesn't work
            d[game_index] = game_year

            #gets last key in dictionary (which is the index of the game in our case)
            game_index = next(reversed(d.keys()))
            game_index = int(game_index)
            

            df1["release_date"][game_index] = game_year
            
            
            print(url)
            print(game_year)
        
        except:  
            # 3) if there is text and then a table under it, select both and search for date in both
            try:
                all_text = soup.find("div", class_="Crs1tb").text.strip()
                unfiltered_years_list = re.findall('\d+', all_text)
                for i in range(len(unfiltered_years_list)):
                    unfiltered_years_list[i] = int(unfiltered_years_list[i])

                # filters numbers between 1930 and 2025
                for i in range(10):
                    for j in unfiltered_years_list:
                        if j < 1930 or j >= 2025:
                            unfiltered_years_list.remove(j)
                game_year = max(unfiltered_years_list)
                
                #use game index to put in dictionary with the date found from scraping, this is so we can replace the new date in df
                #for whatever reason it seems that i can only replace the data of the row if i do it manually or if it is in a dictionary, having it in an array doesn't work
                d[game_index] = game_year

                #gets last key in dictionary (which is the index of the game in our case)
                game_index = next(reversed(d.keys()))
                game_index = int(game_index)

                df1["release_date"][game_index] = game_year                

                print(url)
                print(game_year)

            except:
                try:
                    # 4) if there is a big table different style from top 
                    all_text = soup.find("div", class_="EDblX").text.strip()
                    unfiltered_years_list = re.findall('\d+', all_text)
                    for i in range(len(unfiltered_years_list)):
                        unfiltered_years_list[i] = int(unfiltered_years_list[i])

                    for i in range(10):
                        for j in unfiltered_years_list:
                            if j < 1930 or j >= 2025:
                                unfiltered_years_list.remove(j)
                    game_year = max(unfiltered_years_list)
                    
                    #use game index to put in dictionary with the date found from scraping, this is so we can replace the new date in df
                    #for whatever reason it seems that i can only replace the data of the row if i do it manually or if it is in a dictionary, having it in an array doesn't work
                    d[game_index] = game_year

                    #gets last key in dictionary (which is the index of the game in our case)
                    game_index = next(reversed(d.keys()))
                    game_index = int(game_index)

                    df1["release_date"][game_index] = game_year  
                    
                    print(url)
                    print(game_year)

                except:
                    #last resort, sets unfound data as 0
                    df1["release_date"][wrong_data_combined[i][0]] = 0
                    
                    print(url)
                    print("couldnt find it loser")

        
print("DONE")

C:\Users\Ahmed\AppData\Local\Temp/ipykernel_14584/1763191433.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["release_date"][game_index] = game_year


https://www.google.com/search?q=video game Total War: Three Kingdoms release date
2019
https://www.google.com/search?q=video game Deliver Us the Moon release date
2018
https://www.google.com/search?q=video game Super Street Fighter IV: 3D Edition release date
2010
https://www.google.com/search?q=video game Jurassic Park release date
2011


C:\Users\Ahmed\AppData\Local\Temp/ipykernel_14584/1763191433.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["release_date"][game_index] = game_year


https://www.google.com/search?q=video game Squadron 42 release date
2022
https://www.google.com/search?q=video game Cyberpunk 2077 - Phantom Liberty release date
2023
https://www.google.com/search?q=video game The Lord of the Rings: Gollum release date
2023
https://www.google.com/search?q=video game Star Citizen release date
2014
https://www.google.com/search?q=video game Off the Grid release date
2023
https://www.google.com/search?q=video game Joyride Universe release date
2022
https://www.google.com/search?q=video game Silent Hill 2 release date
2001
https://www.google.com/search?q=video game Kingdom Hearts IV release date
2024
https://www.google.com/search?q=video game Star Wars: Eclipse release date
2024


C:\Users\Ahmed\AppData\Local\Temp/ipykernel_14584/1763191433.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["release_date"][wrong_data_combined[i][0]] = 0


...

C:\Users\Ahmed\AppData\Local\Temp/ipykernel_14584/1763191433.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["release_date"][game_index] = game_year


https://www.google.com/search?q=video game Ion release date
2019


C:\Users\Ahmed\AppData\Local\Temp/ipykernel_14584/1763191433.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["release_date"][wrong_data_combined[i][0]] = 0


https://www.google.com/search?q=video game Never Say Die release date
couldnt find it loser
https://www.google.com/search?q=video game Devil's Liminal release date
2023
DONE


C:\Users\Ahmed\AppData\Local\Temp/ipykernel_14584/1763191433.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["release_date"][game_index] = game_year


In [100]:
#Save CSV after release date cleaning 
df1.to_csv('CSVs/3) releaseDateCleaned.csv', index=False)

In [101]:
#after cleaning releaseDate
df1 = pd.read_csv("CSVs/3) releaseDateCleaned.csv")
df1

,title,release_date,maturity_rating,genre,rating,directors,stars,votes
0,The Witcher 3: Wild Hunt - Blood and Wine,2016,M,['Action' ' Adventure' ' Drama'],9.8,['Konrad Tomaszkiewicz'],['Doug Cockle' ' MyAnna Buring' ' Antonia Bern...,8795
1,The Last of Us,2013,M,['Action' ' Adventure' ' Drama'],9.7,['Neil Druckmann' ' Bruce Straley'],['Troy Baker' ' Ashley Johnson' ' Hana Hayes' ...,69017
2,God of War: Ragnarök,2022,M,['Action' ' Adventure' ' Drama'],9.7,['Eric Williams'],['Christopher Judge' ' Sunny Suljic' ' Daniell...,11569
3,Red Dead Redemption II,2018,M,['Action' ' Adventure' ' Crime'],9.7,NaN,['Benjamin Byron Davis' ' Roger Clark' ' Rob W...,42853
4,The Witcher 3: Wild Hunt,2015,M,['Action' ' Adventure' ' Drama'],9.7,['Konrad Tomaszkiewicz'],['Doug Cockle' ' Denise Gough' ' Jo Wyatt' ' J...,29120
...,...,...,...,...,...,...,...,...
33504,Through the Winter,2023,NaN,NaN,0.0,['Michael Smith'],['Rita Amparita' ' Hannah Beard' ' Xander M Gr...,0
33505,Atari 50: The Anniversary Celebration,2022,NaN,['Documentary' ' Action' ' Adventure'],0.0,NaN,['Al Alcorn' ' Nolan Bushnell' ' David Crane'],0
33506,Yars Revenge Enhanced,2022,NaN,['Action' ' Fantasy' ' Sci-Fi'],0.0,NaN,NaN,0
33507,Our Summers,2022,NaN,Romance,0.0,['Wavi'],['Reece Bridger'],0


In [102]:
#Check what data is still wrong

#gets index of wrong date data
wrong_date_index = np.array(list(df1.index[df1["release_date"] == 0]))
#gets title of wrong date data
wrong_date_title = np.array(list(df1["title"].loc[df1["release_date"] == 0]))

#puts index besides the name of game
wrong_data_combined = np.stack((wrong_date_index, wrong_date_title), axis=1)

print(f"Length: {len(wrong_data_combined)}")
print()
print(wrong_data_combined)

Length: 282

[['14427' 'Beyond Good & Evil 2']
 ['14466' 'Hellraid']
 ['14483' 'Prince of Persia: The Sands of Time Remake']
 ['14504' 'Judas']
 ['14511' 'Ascension Chronicles: Spheres of Influence']
 ['14533' 'MegaRace: DeathMatch']
 ['14748' 'I Married a Monster on a Hill']
 ['14756' "Assassin's Creed: Codename Red"]
 ['14783' 'EXP: War Trauma']
 ['14826' "Camp's Not Dead"]
 ['14848' 'The Witcher Remake']
 ['14939' 'Little Devil Inside']
 ['15176' 'EyesOnly - TopSecret']
 ['15380' 'Doragonbôru Z: Sparking! 4']
 ['15414' 'Avowed']
 ['15488' 'Text Me Back']
 ['15519' 'Echoes of the End']
 ['15603' 'Doomsday Paradise']
 ['15635' 'Fiona Frightening and the Wicked Wardrobe']
 ['15763' 'Fuzzy Gets a House']
 ['15787' 'Enenra']
 ['15820' 'Where Winds Meet']
 ['15853' 'PixelWave Island']
 ['15874' 'Quest Failed - Chapter Two']
 ['15890' 'Midautumn']
 ['15933' 'My Eyes on You']
 ['15945' 'Rat Race']
 ['15953' 'Last Days of Snow']
 ['16256' 'Brassheart']
 ['16348'
  "Jamjam'n Jelly - The nonse

<b> Cleans the release_date data using Steam and validates it compare it to the data collected from google

In [23]:
#going to open the unclean date CSV file to find all the data that was wrong again
df1 = pd.read_csv("CSVs/2) titleCleaned.csv")
#going to open the mostly cleaned CSV file, and going to compare the Google collected data with the Steam collected data
googled_df = pd.read_csv("CSVs/3) releaseDateCleaned.csv")

#this will be the dataframe that has the compared data of the Steam and Google Dataframes
final_df = googled_df.copy()

In [3]:
#gets index of wrong date data
wrong_date_index = np.array(list(df1.index[df1["release_date"] == 0]))
#gets title of wrong date data
wrong_date_title = np.array(list(df1["title"].loc[df1["release_date"] == 0]))

#puts index besides the name of game
wrong_data_combined = np.stack((wrong_date_index, wrong_date_title), axis=1)

print(f"Length: {len(wrong_data_combined)}")
print()
print(wrong_data_combined)

Length: 612

[['4169' 'Total War: Three Kingdoms']
 ['5151' 'Deliver Us the Moon']
 ['5638' 'Super Street Fighter IV: 3D Edition']
 ...
 ['33419' 'Ion']
 ['33437' 'Never Say Die']
 ['33498' "Devil's Liminal"]]


In [50]:
d = {}

for i in range(len(df1["title"].loc[df1["release_date"] == 0])):
    game_name = wrong_data_combined[i][1]
    searchterm = game_name # edit me

    url = r"https://store.steampowered.com/search/?term=" + searchterm #will put what I want to search after q=
    driver = webdriver.Chrome()
    driver.get(url)
    content = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(content,"html.parser")
    
    #will be used downwards to find if the year that steam  souped is equal to year found by google
    int_index = int(wrong_data_combined[i][0])
    
    
    try:
        #special characters that need to be removed from the game title
        special_characters = ["™", "®"]
        
        #going to soup the name of the first title that pops to see if the souped game is the same game from the dataframe or not
        game_souped_name = soup.find("div", class_="col search_name ellipsis").text.strip()
        
        #removes any special characters if needed
        for character in special_characters:
            game_souped_name = game_souped_name.replace(character, "")
        
        if game_souped_name.lower() == (wrong_date_title[i]).lower():
            unfiltered_game_year = soup.find("div", class_="search_released").text.strip()
            unfiltered_years_list = re.findall('\d+', unfiltered_game_year)

            #changes the values inside list into integers
            for i in range(len(unfiltered_years_list)):
                unfiltered_years_list[i] = int(unfiltered_years_list[i])

            #filters numbers between 1930
            for k in range(10):
                for j in unfiltered_years_list:
                    if j < 1930:
                        unfiltered_years_list.remove(j)
            game_year = unfiltered_years_list

            #if year found by steam is equal to year found by google, do nothing
            if game_year[0] == googled_df["release_date"][int_index]:
                print("Same Year")

            #if year found by steam isnt the same found by google, replace the google one with the 
            else:
                print("Changed Date successfully")
                final_df["release_date"].iloc[int_index] = game_year[0]

            print(f"Steam: {game_year[0]}")
            print(f"Google: {googled_df['release_date'][int_index]}")

        #if it does not share the same title, it isnt the same game
        else:
            print("not same game")

        print(f"STEAM: {game_souped_name}")
        print(f"Google: {googled_df['title'][int_index]}")
        print(url)
    
    except:
        print("No Game Found")
        print(f"STEAM: {game_souped_name}")
        print(f"Google: {googled_df['title'][int_index]}")
        print(url)

        
        
    print("-----------------------------------------------")

Same Year
Steam: 2019
Google: 2019
STEAM: Total War: THREE KINGDOMS
Google: Total War: Three Kingdoms
https://store.steampowered.com/search/?term=Total War: Three Kingdoms
-----------------------------------------------
Changed Date successfully
Steam: 2019
Google: 2018
STEAM: Deliver Us The Moon
Google: Deliver Us the Moon
https://store.steampowered.com/search/?term=Deliver Us the Moon
-----------------------------------------------


C:\Users\Ahmed\AppData\Local\Temp/ipykernel_20788/1684921153.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["release_date"].iloc[int_index] = game_year[0]


not same game
STEAM: Ultra Street Fighter IV
Google: Super Street Fighter IV: 3D Edition
https://store.steampowered.com/search/?term=Super Street Fighter IV: 3D Edition
-----------------------------------------------
not same game
STEAM: Jurassic World Evolution: Jurassic Park Edition
Google: Jurassic Park
https://store.steampowered.com/search/?term=Jurassic Park
-----------------------------------------------
not same game
STEAM: War Thunder - British Starter Pack
Google: Squadron 42
https://store.steampowered.com/search/?term=Squadron 42
-----------------------------------------------
No Game Found
STEAM: War Thunder - British Starter Pack
Google: Cyberpunk 2077 - Phantom Liberty
https://store.steampowered.com/search/?term=Cyberpunk 2077 - Phantom Liberty
-----------------------------------------------
Same Year
Steam: 2023
Google: 2023
STEAM: The Lord of the Rings: Gollum
Google: The Lord of the Rings: Gollum
https://store.steampowered.com/search/?term=The Lord of the Rings: Gollum


No Game Found
STEAM: Ratten Reich
Google: DNF2001 Restoration Project
https://store.steampowered.com/search/?term=DNF2001 Restoration Project
-----------------------------------------------
No Game Found
STEAM: Ratten Reich
Google: I Married a Monster on a Hill
https://store.steampowered.com/search/?term=I Married a Monster on a Hill
-----------------------------------------------
not same game
STEAM: Assassin’s Creed Chronicles: China
Google: Assassin's Creed: Codename Red
https://store.steampowered.com/search/?term=Assassin's Creed: Codename Red
-----------------------------------------------
No Game Found
STEAM: EXP: War Trauma
Google: EXP: War Trauma
https://store.steampowered.com/search/?term=EXP: War Trauma
-----------------------------------------------
not same game
STEAM: Interfectorem
Google: Camp's Not Dead
https://store.steampowered.com/search/?term=Camp's Not Dead
-----------------------------------------------
not same game
STEAM: The Orchard of Stray Sheep
Google: Projec

not same game
STEAM: My Eyes
Google: My Eyes on You
https://store.steampowered.com/search/?term=My Eyes on You
-----------------------------------------------
not same game
STEAM: Rat Racer
Google: Rat Race
https://store.steampowered.com/search/?term=Rat Race
-----------------------------------------------
not same game
STEAM: Six Days of Snow
Google: Last Days of Snow
https://store.steampowered.com/search/?term=Last Days of Snow
-----------------------------------------------
not same game
STEAM: Remothered: Tormented Fathers
Google: Remothered: Chapter 3
https://store.steampowered.com/search/?term=Remothered: Chapter 3
-----------------------------------------------
Same Year
Steam: 2018
Google: 2018
STEAM: Crimson Keep
Google: Crimson Keep
https://store.steampowered.com/search/?term=Crimson Keep
-----------------------------------------------
No Game Found
STEAM: Eternights
Google: Eternights
https://store.steampowered.com/search/?term=Eternights
------------------------------------

...

not same game
STEAM: Masks Of Deception
Google: Demon's Deception
https://store.steampowered.com/search/?term=Demon's Deception
-----------------------------------------------
No Game Found
STEAM: Lovely Plains
Google: Lovely Plains
https://store.steampowered.com/search/?term=Lovely Plains
-----------------------------------------------
Same Year
Steam: 2018
Google: 2018
STEAM: POSTWORLD
Google: Postworld
https://store.steampowered.com/search/?term=Postworld
-----------------------------------------------
No Game Found
STEAM: POSTWORLD
Google: Tri-City RP
https://store.steampowered.com/search/?term=Tri-City RP
-----------------------------------------------
No Game Found
STEAM: POSTWORLD
Google: Wuchang: Fallen Feathers
https://store.steampowered.com/search/?term=Wuchang: Fallen Feathers
-----------------------------------------------
No Game Found
STEAM: POSTWORLD
Google: Age of Aincrad
https://store.steampowered.com/search/?term=Age of Aincrad
----------------------------------------

No Game Found
STEAM: The Store is Closed
Google: The Store Is Closed
https://store.steampowered.com/search/?term=The Store Is Closed
-----------------------------------------------
No Game Found
STEAM: Desolation Seed
Google: Desolation Seed
https://store.steampowered.com/search/?term=Desolation Seed
-----------------------------------------------
not same game
STEAM: Maggie's Movies - Camera, Action!
Google: Camera 6
https://store.steampowered.com/search/?term=Camera 6
-----------------------------------------------
No Game Found
STEAM: Maggie's Movies - Camera, Action!
Google: EverQuest Next
https://store.steampowered.com/search/?term=EverQuest Next
-----------------------------------------------
Same Year
Steam: 2023
Google: 2023
STEAM: The Orphanage
Google: The Orphanage
https://store.steampowered.com/search/?term=The Orphanage
-----------------------------------------------
No Game Found
STEAM: The Orphanage
Google: Pithon
https://store.steampowered.com/search/?term=Pithon
-------

not same game
STEAM: Reversed Front
Google: Operation Stabilization: Revived
https://store.steampowered.com/search/?term=Operation Stabilization: Revived
-----------------------------------------------
not same game
STEAM: Sekiro: Shadows Die Twice - GOTY Edition
Google: Softwar
https://store.steampowered.com/search/?term=Softwar
-----------------------------------------------
No Game Found
STEAM: Sekiro: Shadows Die Twice - GOTY Edition
Google: FNAF World DX
https://store.steampowered.com/search/?term=FNAF World DX
-----------------------------------------------
No Game Found
STEAM: Sekiro: Shadows Die Twice - GOTY Edition
Google: Astropal
https://store.steampowered.com/search/?term=Astropal
-----------------------------------------------
not same game
STEAM: Vidar
Google: Vidar: The RPG Where Everyone Dies
https://store.steampowered.com/search/?term=Vidar: The RPG Where Everyone Dies
-----------------------------------------------
not same game
STEAM: The Quarry – Deluxe Bonus Conten

not same game
STEAM: Hoser Hockey
Google: Hockey
https://store.steampowered.com/search/?term=Hockey
-----------------------------------------------
No Game Found
STEAM: Hoser Hockey
Google: Dead Life Necrodevils
https://store.steampowered.com/search/?term=Dead Life Necrodevils
-----------------------------------------------
not same game
STEAM: Falcon Gold
Google: Video Gold
https://store.steampowered.com/search/?term=Video Gold
-----------------------------------------------
No Game Found
STEAM: Falcon Gold
Google: Kyle Rittenhouse's Turkey Shoot
https://store.steampowered.com/search/?term=Kyle Rittenhouse's Turkey Shoot
-----------------------------------------------
not same game
STEAM: Knights of the Sky
Google: Bust My Balloons
https://store.steampowered.com/search/?term=Bust My Balloons
-----------------------------------------------
not same game
STEAM: Lost in Play
Google: Challenger in Between
https://store.steampowered.com/search/?term=Challenger in Between
------------------

not same game
STEAM: The Ditzy Demons Are in Love With Me
Google: In Lukewarm Blood
https://store.steampowered.com/search/?term=In Lukewarm Blood
-----------------------------------------------
No Game Found
STEAM: The Ditzy Demons Are in Love With Me
Google: The Louisa Williamson Hutchison Collection: ULV Cultural & Natural History Collections
https://store.steampowered.com/search/?term=The Louisa Williamson Hutchison Collection: ULV Cultural & Natural History Collections
-----------------------------------------------
No Game Found
STEAM: The Ditzy Demons Are in Love With Me
Google: Prisionero de Oubliette (Realidad Virtual)
https://store.steampowered.com/search/?term=Prisionero de Oubliette (Realidad Virtual)
-----------------------------------------------
No Game Found
STEAM: The Ditzy Demons Are in Love With Me
Google: SimpliZoo
https://store.steampowered.com/search/?term=SimpliZoo
-----------------------------------------------
not same game
STEAM: DOA
Google: Dead on Arrival
htt

In [ ]:
# Save CSV after using Steam for cleaning the release_date
final_df.to_csv('CSVs/4) finalReleaseDateCleaned.csv', index=False)

In [7]:
final_df = pd.read_csv("CSVs/4) finalReleaseDateCleaned.csv")

In [8]:
#Check what data is still wrong
wrong_date_index = np.array(list(final_df.index[final_df["release_date"] == 0]))
#gets title of wrong date data
wrong_date_title = np.array(list(final_df["title"].loc[final_df["release_date"] == 0]))

#puts index besides the name of game
wrong_data_combined = np.stack((wrong_date_index, wrong_date_title), axis=1)

print(f"Length: {len(wrong_data_combined)}")
print()
print(wrong_data_combined)

Length: 269

[['14427' 'Beyond Good & Evil 2']
 ['14466' 'Hellraid']
 ['14483' 'Prince of Persia: The Sands of Time Remake']
 ['14504' 'Judas']
 ['14511' 'Ascension Chronicles: Spheres of Influence']
 ['14533' 'MegaRace: DeathMatch']
 ['14748' 'I Married a Monster on a Hill']
 ['14756' "Assassin's Creed: Codename Red"]
 ['14783' 'EXP: War Trauma']
 ['14826' "Camp's Not Dead"]
 ['14848' 'The Witcher Remake']
 ['14939' 'Little Devil Inside']
 ['15176' 'EyesOnly - TopSecret']
 ['15380' 'Doragonbôru Z: Sparking! 4']
 ['15414' 'Avowed']
 ['15488' 'Text Me Back']
 ['15519' 'Echoes of the End']
 ['15635' 'Fiona Frightening and the Wicked Wardrobe']
 ['15763' 'Fuzzy Gets a House']
 ['15787' 'Enenra']
 ['15820' 'Where Winds Meet']
 ['15853' 'PixelWave Island']
 ['15874' 'Quest Failed - Chapter Two']
 ['15933' 'My Eyes on You']
 ['15945' 'Rat Race']
 ['15953' 'Last Days of Snow']
 ['16256' 'Brassheart']
 ['16348'
  "Jamjam'n Jelly - The nonsensical, high tactical, 110% cute em up."]
 ['16515' 'L

<b> Just going to Delete the rest of the incorrect data as most of them are unreleased games or games I cant scrap from Google or Steam

In [21]:
for i in range(len(wrong_date_index)):
    final_df = final_df.drop(wrong_date_index[i])
print("done")

done


In [23]:
#check if the data is still there
wrong_date_index = np.array(list(final_df.index[final_df["release_date"] == 0]))
#gets title of wrong date data
wrong_date_title = np.array(list(final_df["title"].loc[final_df["release_date"] == 0]))

#puts index besides the name of game
wrong_data_combined = np.stack((wrong_date_index, wrong_date_title), axis=1)

print(f"Length: {len(wrong_data_combined)}")
print()
print(wrong_data_combined)

Length: 0

[]


In [24]:
#replace old finalReleaseDateCleaned.csv with wrong data to the new one
final_df.to_csv('CSVs/4) finalReleaseDateCleaned.csv', index=False)

<b> a again: forgot to check for duplicated titles so I will do it now

In [70]:
df = pd.read_csv("CSVs/4) finalReleaseDateCleaned.csv")

In [71]:
df.loc[df.duplicated(), :]

,title,release_date,maturity_rating,genre,rating,directors,stars,votes
16554,Basketball Legends,2021,NaN,['Action' ' Adventure' ' Game-Show'],0.0,['Jordan Wilson'],['Jords Iii' ' Jordan Wilson'],0
19826,Tele-Sports,1977,NaN,Sport,0.0,NaN,NaN,0
21810,TV Game,1976,NaN,Sport,0.0,NaN,NaN,0
22197,Mahjong: The Ladyhunter - Kaitô Kuroneko Renme...,1990,NaN,NaN,0.0,NaN,NaN,0
22937,Draw Poker,1982,NaN,NaN,0.0,NaN,NaN,0
24210,Last Action Hero,1994,NaN,Action,0.0,NaN,NaN,0
25038,Draw Poker,1982,NaN,NaN,0.0,NaN,NaN,0
25464,Strike Mission,1984,NaN,Action,0.0,NaN,NaN,0
26167,Draw Poker,1982,NaN,NaN,0.0,NaN,NaN,0
26240,Video Sports,1976,NaN,Sport,0.0,NaN,NaN,0


In [72]:
df = df.drop_duplicates()

In [73]:
#check if there is any duplication again
df.loc[df.duplicated(), :]

,title,release_date,maturity_rating,genre,rating,directors,stars,votes


In [77]:
df.loc[df['title'] == 'Basketball Legends']

,title,release_date,maturity_rating,genre,rating,directors,stars,votes
15550,Basketball Legends,2021,NaN,['Action' ' Adventure' ' Game-Show'],0.0,['Jordan Wilson'],['Jords Iii' ' Jordan Wilson'],0
